In [7]:
import numpy as np
import jieba

# 1. 文本分词
text = "香蕉和樱桃是我最喜欢的水果之一。"
words = list(jieba.lcut(text)) 
words = [w for w in words if w not in ['。']] 
n_words = len(words)
print("分词结果：", words)
print("单词数量：", n_words)

分词结果： ['香蕉', '和', '樱桃', '是', '我', '最', '喜欢', '的', '水果', '之一']
单词数量： 10


In [8]:
# 2. 定义滑动窗口（窗口大小=2，左右各1个邻居）
window_size = 2  # 窗口大小=左右邻居数+1（自身）
adj = np.zeros((n_words, n_words), dtype=int)  # 邻接矩阵 n×n

for i in range(n_words):
    # 窗口范围：[max(0, i-1), min(n_words-1, i+1)]（左右各1个邻居）
    start = max(0, i - (window_size // 2))
    end = min(n_words - 1, i + (window_size // 2))
    for j in range(start, end + 1):
        if i != j:  # 排除自身（可根据需求调整是否包含）
            adj[i][j] = 1
print("邻接矩阵：\n", adj)

cherry_idx = words.index("樱桃")
print("樱桃的索引：", cherry_idx)
print("樱桃的邻居索引：", np.where(adj[cherry_idx] == 1)[0])

邻接矩阵：
 [[0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 1 0]]
樱桃的索引： 2
樱桃的邻居索引： [1 3]


In [9]:
# 3. 初始化参数
embed_dim = 5  # 单词嵌入维度
hidden1_dim = 8  # Layer1隐藏层维度
hidden2_dim = 10  # Layer2隐藏层维度

# 单词嵌入矩阵 X: (n_words, embed_dim)
np.random.seed(42) 
X = np.random.randn(n_words, embed_dim)  # 正态分布初始化

# 权重矩阵与偏置（线性变换参数）
W1 = np.random.randn(embed_dim, hidden1_dim)  # Layer1权重: (embed_dim, hidden1_dim)
B1 = np.random.randn(hidden1_dim)  # Layer1偏置: (hidden1_dim,)
W2 = np.random.randn(hidden1_dim, hidden2_dim)  # Layer2权重: (hidden1_dim, hidden2_dim)
B2 = np.random.randn(hidden2_dim)  # Layer2偏置: (hidden2_dim,)

print("X（单词嵌入）形状：", X.shape)
print("W1形状：", W1.shape, " B1形状：", B1.shape)
print("W2形状：", W2.shape, " B2形状：", B2.shape)

X（单词嵌入）形状： (10, 5)
W1形状： (5, 8)  B1形状： (8,)
W2形状： (8, 10)  B2形状： (10,)


In [4]:
# 4. Layer0：输入层，节点表示=单词嵌入（无聚合）
h0 = X  # h0: (n_words, embed_dim)
h0_cherry = h0[cherry_idx]  # 樱桃的Layer0表示

print("Layer0 樱桃的表示 h0_樱桃：\n", h0_cherry)
print("h0_樱桃形状：", h0_cherry.shape)

Layer0 樱桃的表示 h0_樱桃：
 [-0.46341769 -0.46572975  0.24196227 -1.91328024 -1.72491783]
h0_樱桃形状： (5,)


In [5]:
# 5. Layer1：Mean聚合 + 线性变换 + Sigmoid激活
def mean_aggregate(h_prev, adj, node_idx):
    """Mean聚合函数：计算节点邻居的前一层表示均值"""
    # 找到节点的邻居索引
    neighbor_idxs = np.where(adj[node_idx] == 1)[0]
    # 聚合邻居表示（均值）
    aggregated = np.mean(h_prev[neighbor_idxs], axis=0)
    return aggregated

def sigmoid(x):
    """Sigmoid激活函数（提示图指定）"""
    return 1 / (1 + np.exp(-x))

# 计算樱桃的Layer1表示 h1_cherry
aggregated_h0 = mean_aggregate(h0, adj, cherry_idx)  # 邻居h0均值聚合
h1_cherry = sigmoid(np.dot(aggregated_h0, W1) + B1)  # 线性变换+激活

# 计算所有节点的Layer1表示（可选，用于Layer2聚合）
h1 = np.zeros((n_words, hidden1_dim))
for i in range(n_words):
    agg = mean_aggregate(h0, adj, i)
    h1[i] = sigmoid(np.dot(agg, W1) + B1)

print("Layer1 樱桃的聚合向量（邻居h0均值）：\n", aggregated_h0)
print("Layer1 樱桃的表示 h1_cherry：\n", h1_cherry)
print("h1_cherry形状：", h1_cherry.shape)

Layer1 樱桃的聚合向量（邻居h0均值）：
 [-0.39821224  0.28319085  0.54084103 -0.68874923 -0.43487183]
Layer1 樱桃的表示 h1_cherry：
 [0.76576331 0.83206074 0.47976927 0.36619341 0.20072104 0.471204
 0.77253963 0.7586549 ]
h1_cherry形状： (8,)


In [6]:
# 6. Layer2：Mean聚合 + 线性变换 + Sigmoid激活
# 聚合Layer1中樱桃的邻居表示
aggregated_h1 = mean_aggregate(h1, adj, cherry_idx)  # 邻居h1均值聚合
h2_cherry = sigmoid(np.dot(aggregated_h1, W2) + B2)  # 线性变换+激活

print("Layer2 樱桃的聚合向量（邻居h1均值）：\n", aggregated_h1)
print("Layer2 樱桃的表示 h2_cherry：\n", h2_cherry)
print("h2_cherry形状：", h2_cherry.shape)

Layer2 樱桃的聚合向量（邻居h1均值）：
 [0.60995157 0.6321823  0.53944936 0.61684746 0.32947389 0.67369274
 0.63869334 0.35339258]
Layer2 樱桃的表示 h2_cherry：
 [0.58693537 0.92907665 0.06454757 0.33174692 0.19538224 0.71093774
 0.57496472 0.97394374 0.87689379 0.598684  ]
h2_cherry形状： (10,)
